# Step 15: V12 — SWEET (variability at transit period)

Goal: test whether sinusoidal variability at the transit period (P) or half-period (P/2) could explain the observed signal.

Why this matters:
- Rotational modulation or other stellar variability can produce apparent periodic signals that interact with transit detection/uniqueness tests.
- SWEET is a standard robovetter-style check that fits sinusoids at P, P/2, and 2P.

This check requires the `exovetter` dependency.


In [ ]:
from pathlib import Path
import json
import sys

tutorial_dir = Path('docs/tutorials/tutorial_toi-5807-incremental').resolve()
sys.path.insert(0, str(tutorial_dir))

import toi5807_shared as sh

ds = sh.load_dataset()
lc = sh.stitch_pdcsap(ds)
depth_ppm, _ = sh.estimate_depth_ppm(lc)
candidate = sh.make_candidate(depth_ppm)

session = sh.make_session(stitched=lc, candidate=candidate, network=False)
r = session.run('V12')

print(
    json.dumps(
        {
            'status': r.status,
            'flags': r.flags,
            'metrics': r.metrics,
        },
        indent=2,
        sort_keys=True,
    )
)


<details>
<summary><b>Expected Output</b></summary>

```text
{
  "flags": [],
  "metrics": {
    "baseline_days": 787.27,
    "cadence_median_min": 2.0,
    "n_points": 73805,
    "n_transits_expected": 55,
    "snr_at_period": 4.505338699040724,
    "snr_double_period": 0.37441144768396967,
    "snr_half_period": 4.3168120938420484,
    "sweet_msg": "WARN: SWEET test finds signal at HALF transit period\nWARN: SWEET test finds signal at the transit period"
  },
  "status": "ok"
}
```

</details>


In [ ]:
# Plot V12: SWEET
out = {
    'status': r.status,
    'flags': r.flags,
}

try:
    import matplotlib.pyplot as plt
    from bittr_tess_vetter.api import plot_sweet
    PLOTTING_AVAILABLE = True
except Exception as e:
    PLOTTING_AVAILABLE = False
    out['plotting_error'] = str(e)

if PLOTTING_AVAILABLE and (r.status == 'ok'):
    run_out_dir, docs_out_dir = sh.artifact_dirs(step_id='15_v12_sweet')
    run_path = run_out_dir / 'V12_sweet.png'
    docs_path = (docs_out_dir / 'V12_sweet.png') if docs_out_dir is not None else None

    fig, ax = plt.subplots(figsize=(8, 5))
    plot_sweet(r, ax=ax)
    ax.set_title('V12: SWEET (variability at transit period)')
    fig.tight_layout()

    fig.savefig(run_path, dpi=150, bbox_inches='tight')
    if docs_path is not None:
        fig.savefig(docs_path, dpi=150, bbox_inches='tight')
    plt.show()

    out['run_plot_path'] = str(run_path)
    out['docs_plot_path'] = str(docs_path) if docs_path is not None else None

print(json.dumps(out, indent=2, sort_keys=True))


**Pre-rendered plot (no execution required):** `../artifacts/tutorial_toi-5807-incremental/15_v12_sweet/V12_sweet.png`

![V12: SWEET](../artifacts/tutorial_toi-5807-incremental/15_v12_sweet/V12_sweet.png)


<details>
<summary><b>Expected Output (plot cell)</b></summary>

```text
{
  "docs_plot_path": "docs/tutorials/artifacts/tutorial_toi-5807-incremental/15_v12_sweet/V12_sweet.png",
  "flags": [],
  "run_plot_path": "persistent_cache/tutorial_toi-5807-incremental/15_v12_sweet/V12_sweet.png",
  "status": "ok"
}
```

</details>


<details>
<summary><b>Analysis</b></summary>

- **Flags:** none (but `sweet_msg` contains WARN lines).
- **Result:** SWEET reports SNR ∼4.5 at P and ∼4.3 at P/2 (2P is small).
- **Interpretation:** suggests some coherent variability/systematics at P and P/2; treat as a warning that motivates per-sector and/or detrended reruns before drawing strong uniqueness conclusions.
- **Next step:** proceed to false-alarm checks (V13 gaps, V15 asymmetry) and then re-run V11/V12 after any detrending/sector gating.

</details>
